In [1]:
import os
import pandas as pd
import requests
import json
import yfinance as yf
from dotenv import load_dotenv
from openai import OpenAI
import pprint


In [2]:
# Load .env file
load_dotenv()

True

In [3]:
# Check if the environment variable is set for Open AI API
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("The environment variable 'OPENAI_API_KEY' is not set.")

In [4]:
# Check if the environment variable is set for Vantage API
van_api_key = os.getenv("VANTAGE_API_KEY")
if van_api_key is None:
    raise ValueError("The environment variable 'VANTAGE_API_KEY' is not set.")

In [5]:
# Stock symbol for S&P 500
symbol = "SPY"

# API URL for real-time stock data
stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={van_api_key}"

# Fetch stock data
response = requests.get(stock_url)

stock_data = response.json()

print(stock_data)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'SPY', '3. Last Refreshed': '2025-03-10', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-03-10': {'1. open': '567.5900', '2. high': '569.5400', '3. low': '555.5900', '4. close': '560.5800', '5. volume': '97084529'}, '2025-03-07': {'1. open': '570.9000', '2. high': '577.3900', '3. low': '565.6300', '4. close': '575.9200', '5. volume': '81158816'}, '2025-03-06': {'1. open': '575.4800', '2. high': '580.1736', '3. low': '570.1200', '4. close': '572.7100', '5. volume': '80094868'}, '2025-03-05': {'1. open': '576.6900', '2. high': '584.8800', '3. low': '573.0795', '4. close': '583.0600', '5. volume': '71230528'}, '2025-03-04': {'1. open': '579.7100', '2. high': '585.3900', '3. low': '572.2500', '4. close': '576.8600', '5. volume': '109648212'}, '2025-03-03': {'1. open': '596.1800', '2. high': '597.3400', '3. low': '579.9000', '4. close': '583.7700', '5

In [6]:
# Extract time series data
time_series = stock_data["Time Series (Daily)"]
print(time_series)

{'2025-03-10': {'1. open': '567.5900', '2. high': '569.5400', '3. low': '555.5900', '4. close': '560.5800', '5. volume': '97084529'}, '2025-03-07': {'1. open': '570.9000', '2. high': '577.3900', '3. low': '565.6300', '4. close': '575.9200', '5. volume': '81158816'}, '2025-03-06': {'1. open': '575.4800', '2. high': '580.1736', '3. low': '570.1200', '4. close': '572.7100', '5. volume': '80094868'}, '2025-03-05': {'1. open': '576.6900', '2. high': '584.8800', '3. low': '573.0795', '4. close': '583.0600', '5. volume': '71230528'}, '2025-03-04': {'1. open': '579.7100', '2. high': '585.3900', '3. low': '572.2500', '4. close': '576.8600', '5. volume': '109648212'}, '2025-03-03': {'1. open': '596.1800', '2. high': '597.3400', '3. low': '579.9000', '4. close': '583.7700', '5. volume': '74249199'}, '2025-02-28': {'1. open': '585.5600', '2. high': '594.7200', '3. low': '582.4400', '4. close': '594.1800', '5. volume': '88744106'}, '2025-02-27': {'1. open': '596.8500', '2. high': '598.0200', '3. lo

In [7]:
# Convert to DataFrame
df = pd.DataFrame.from_dict(time_series, orient="index")
df

,1. open,2. high,3. low,4. close,5. volume
2025-03-10,567.5900,569.5400,555.5900,560.5800,97084529
2025-03-07,570.9000,577.3900,565.6300,575.9200,81158816
2025-03-06,575.4800,580.1736,570.1200,572.7100,80094868
2025-03-05,576.6900,584.8800,573.0795,583.0600,71230528
2025-03-04,579.7100,585.3900,572.2500,576.8600,109648212
...,...,...,...,...,...
2024-10-18,584.0700,585.3900,582.5800,584.5900,37416801
2024-10-17,585.9100,586.1200,582.1600,582.3500,34393714
2024-10-16,579.7800,582.8300,578.9600,582.3000,30725436
2024-10-15,584.5900,584.9000,578.5450,579.7800,54203636


In [8]:
# Rename columns for readability
df.columns = ["open", "high", "low", "close", "volume"]
df

,open,high,low,close,volume
2025-03-10,567.5900,569.5400,555.5900,560.5800,97084529
2025-03-07,570.9000,577.3900,565.6300,575.9200,81158816
2025-03-06,575.4800,580.1736,570.1200,572.7100,80094868
2025-03-05,576.6900,584.8800,573.0795,583.0600,71230528
2025-03-04,579.7100,585.3900,572.2500,576.8600,109648212
...,...,...,...,...,...
2024-10-18,584.0700,585.3900,582.5800,584.5900,37416801
2024-10-17,585.9100,586.1200,582.1600,582.3500,34393714
2024-10-16,579.7800,582.8300,578.9600,582.3000,30725436
2024-10-15,584.5900,584.9000,578.5450,579.7800,54203636


In [9]:
# Reset index to make the date a column
df = df.reset_index()

In [10]:
# Rename the new column to "date" (if it’s not automatically named)
df = df.rename(columns={"index": "date"})
df

,date,open,high,low,close,volume
0,2025-03-10,567.5900,569.5400,555.5900,560.5800,97084529
1,2025-03-07,570.9000,577.3900,565.6300,575.9200,81158816
2,2025-03-06,575.4800,580.1736,570.1200,572.7100,80094868
3,2025-03-05,576.6900,584.8800,573.0795,583.0600,71230528
4,2025-03-04,579.7100,585.3900,572.2500,576.8600,109648212
...,...,...,...,...,...,...
95,2024-10-18,584.0700,585.3900,582.5800,584.5900,37416801
96,2024-10-17,585.9100,586.1200,582.1600,582.3500,34393714
97,2024-10-16,579.7800,582.8300,578.9600,582.3000,30725436
98,2024-10-15,584.5900,584.9000,578.5450,579.7800,54203636


In [11]:
# Convert to datetime format (recommended)
df["date"] = pd.to_datetime(df["date"])

# Display the DataFrame
df

,date,open,high,low,close,volume
0,2025-03-10,567.5900,569.5400,555.5900,560.5800,97084529
1,2025-03-07,570.9000,577.3900,565.6300,575.9200,81158816
2,2025-03-06,575.4800,580.1736,570.1200,572.7100,80094868
3,2025-03-05,576.6900,584.8800,573.0795,583.0600,71230528
4,2025-03-04,579.7100,585.3900,572.2500,576.8600,109648212
...,...,...,...,...,...,...
95,2024-10-18,584.0700,585.3900,582.5800,584.5900,37416801
96,2024-10-17,585.9100,586.1200,582.1600,582.3500,34393714
97,2024-10-16,579.7800,582.8300,578.9600,582.3000,30725436
98,2024-10-15,584.5900,584.9000,578.5450,579.7800,54203636


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    100 non-null    datetime64[ns]
 1   open    100 non-null    object        
 2   high    100 non-null    object        
 3   low     100 non-null    object        
 4   close   100 non-null    object        
 5   volume  100 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 4.8+ KB


In [13]:
df.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [14]:
df.describe()

,date
count,100
mean,2024-12-25 00:00:00
min,2024-10-14 00:00:00
25%,2024-11-17 06:00:00
50%,2024-12-23 12:00:00
75%,2025-01-31 18:00:00
max,2025-03-10 00:00:00
